## Knowledge Graph Pre-processing

In [ ]:
import json
from tqdm import tqdm
triplets = {}
# Function to convert text file to JSONL
def convert_text_to_jsonl(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in tqdm(infile):
            # Assuming the ID and description are separated by a space
            # Modify the delimiter as needed
            parts = line.strip().split()
            if len(parts) == 3:
                h, r, t = parts
                if not triplets.get(h):
                    triplets[h] = [[h,r,t]]
                else:
                    triplets[h].append([h,r,t])
        for key in triplets:
            json_obj = {'key': key, 'triplets': triplets[key]}
            outfile.write(json.dumps(json_obj) + '\n')

# Replace 'input.txt' with your input file name
# Replace 'output.jsonl' with your desired output file name
convert_text_to_jsonl('/home/zho1rng/oven_train/dataset/wikidata5m/wikidata5m_all_triplet.txt', '/home/zho1rng/oven_train/dataset/wikidata5m/wikidata5m_all_triplet.jsonl')

21354359it [01:16, 280219.28it/s]


In [13]:
import json
entity_set = set()

files = ['/home/zho1rng/oven_train/dataset/oven_data/oven_entity_train.jsonl', 
         '/home/zho1rng/oven_train/dataset/oven_data/oven_entity_val.jsonl', 
         '/home/zho1rng/oven_train/dataset/oven_data/oven_query_train.jsonl', 
         '/home/zho1rng/oven_train/dataset/oven_data/oven_query_val.jsonl', 
         '/home/zho1rng/oven_train/dataset/test_data/oven_entity_test.jsonl', 
         '/home/zho1rng/oven_train/dataset/test_data/oven_query_test.jsonl', 
         '/home/zho1rng/oven_train/dataset/test_data/oven_human.jsonl']

for path in files:
    with open(path, 'r') as f:
        for line in f:
            json_object = json.loads(line)
            q_id = json_object['entity_id']
            entity_set.add(q_id)

print(len(entity_set))

            
with open('missing_entity.txt', 'w') as output:
    for e in entity_set:
        if not triplets.get(e):
            output.write(e + '\n')
    
print(len(triplets))


20549
4799312


In [14]:
with open('oven_entity.txt', 'w') as output:
    for e in entity_set:
            output.write(e + '\n')

In [15]:
with open('wikidata5m_entity.txt', 'w') as output:
    for key in triplets:
            output.write(key + '\n')

In [ ]:
from tqdm import tqdm
with open('/home/zho1rng/oven_train/dataset/wikidata5m+/wikidata5m_relation.txt', 'r') as infile, open('/home/zho1rng/oven_train/dataset/wikidata5m+/wikidata5m_relation+.txt', 'w') as outfile:
    for f in tqdm(infile):
        words = f.split()
        outfile.writelines(words[0] + '\n')

### Triplets with head index to triplets with tail index

In [53]:
from tqdm import tqdm
import json
triplets_path = '/home/zho1rng/oven_train/dataset/wikidata5m+/wikidata5m_all_triplet+.jsonl'
triplets_t_path = '/home/zho1rng/oven_train/dataset/wikidata5m+/wikidata5m_triplet_t+.jsonl'
triplets_h_path = '/home/zho1rng/oven_train/dataset/wikidata5m+/wikidata5m_triplet_h+.jsonl'
entity_path = '/home/zho1rng/oven_train/dataset/wikidata5m+/wikidata5m_entity++.txt'
triplets_list = []
entity_set = set()
with open(entity_path, 'r') as infile:
    for line in tqdm(infile):
        entity_set.add(line[:-1])

with open(triplets_path, 'r') as infile, open(triplets_h_path, 'w') as outfile:
    for line in tqdm(infile):
        json_object = json.loads(line)
        for i in json_object['triplets']:
            triplets_list.append(i)
        if json_object['triplets']!=[]:
            outfile.write(json.dumps(json_object) + '\n')

triplets_dict = {}
for entity in tqdm(entity_set):
    triplets_dict[entity] = []
    
for triplet in tqdm(triplets_list):
    h = triplet[0]
    r = triplet[1]
    t= triplet[2]
    triplets_dict[t].append(triplet)

with open(triplets_t_path, 'w') as outfile:
    for key in tqdm(triplets_dict):
        if triplets_dict[key]!=[]:
            json_obj = {'key': key, 'triplets': triplets_dict[key]}
            outfile.write(json.dumps(json_obj) + '\n')
# with open(entity_path_n, 'w') as outfile:
#     for e in tqdm(entity_set):
#         outfile.writelines(e + '\n')
        

4820473it [00:02, 2399060.86it/s]
4801546it [01:21, 58764.59it/s] 
100%|██████████| 4820473/4820473 [00:23<00:00, 207048.86it/s]


### Get Sub-graph

#### 1-hop

In [54]:
from tqdm import tqdm
kg_path = '../dataset/wikidata5m+/'
sub_kg_path = '../dataset/wikidata5m+_1hop'
import json
oven_entity_set = set()

files = ['/home/zho1rng/oven_train/dataset/oven_data/oven_entity_train.jsonl', 
         '/home/zho1rng/oven_train/dataset/oven_data/oven_entity_val.jsonl', 
         '/home/zho1rng/oven_train/dataset/oven_data/oven_query_train.jsonl', 
         '/home/zho1rng/oven_train/dataset/oven_data/oven_query_val.jsonl', 
         '/home/zho1rng/oven_train/dataset/test_data/oven_entity_test.jsonl', 
         '/home/zho1rng/oven_train/dataset/test_data/oven_query_test.jsonl', 
         '/home/zho1rng/oven_train/dataset/test_data/oven_human.jsonl']

for path in files:
    with open(path, 'r') as f:
        for line in tqdm(f):
            json_object = json.loads(line)
            q_id = json_object['entity_id']
            oven_entity_set.add(q_id)

print(len(oven_entity_set))


rel_set = set()
with open('/home/zho1rng/oven_train/dataset/wikidata5m+/wikidata5m_relation+.txt', 'r') as infile:
    for line in tqdm(infile):
        rel_set.add(line[:-1])
        
triplets_h_dict = {}
triplets_t_dict = {}

with open('/home/zho1rng/oven_train/dataset/wikidata5m+/wikidata5m_triplet_h+.jsonl', 'r') as infile:
    for line in tqdm(infile):
        json_object = json.loads(line)
        triplets_h_dict[json_object['key']] = json_object['triplets']

with open('/home/zho1rng/oven_train/dataset/wikidata5m+/wikidata5m_triplet_t+.jsonl', 'r') as infile:
    for line in tqdm(infile):
        json_object = json.loads(line)
        triplets_t_dict[json_object['key']] = json_object['triplets']


        

4926314it [00:16, 304296.58it/s]
126199it [00:00, 303692.89it/s]
32255it [00:00, 297952.19it/s]
3291it [00:00, 291575.05it/s]
709478it [00:02, 300281.97it/s]
19781it [00:00, 298799.03it/s]
24867it [00:00, 299714.53it/s]


20549


825it [00:00, 1283970.61it/s]
4801110it [00:51, 93453.83it/s] 
153007it [00:19, 7767.31it/s]  


KeyboardInterrupt: 

In [29]:
hop1_entity_set = set()


for e in oven_entity_set:
    if e in triplets_h_dict:
        for triplet in triplets_h_dict[e]:
            hop1_entity_set.add(triplet[0])
            hop1_entity_set.add(triplet[2])
    if e in triplets_t_dict:
        for triplet in triplets_t_dict[e]:
            hop1_entity_set.add(triplet[0])
            hop1_entity_set.add(triplet[2])

print(len(hop1_entity_set))

3506363


#### 0-hop + hierachical info

In [68]:
sub_rel_set= set()
for e in tqdm(oven_entity_set):
    if e in triplets_h_dict:
        for triplet in triplets_h_dict[e]:
            if triplet[2] in oven_entity_set:
                sub_rel_set.add(triplet[1])
    if e in triplets_t_dict:
        for triplet in triplets_t_dict[e]:
            if triplet[0] in oven_entity_set:
                sub_rel_set.add(triplet[1])
                
rel_text_dict = {}
with open('/home/zho1rng/oven_train/dataset/knowledge_base/wikidata_relation.jsonl') as infile:
    for line in infile:
        json_obj = json.loads(line)
        rel_text_dict[json_obj['p_id']] =json_obj['description']

k=50
for rel in sub_rel_set:
    print(rel, rel_text_dict[rel])
    k-=1
    if k ==0:
        break


100%|██████████| 20549/20549 [00:00<00:00, 136725.29it/s]

P360 common element between all listed items
P495 country of origin of this item (creative work, food, phrase, product, etc.)
P832 official public holiday that occurs in this place in its honor, usually a non-working day
P2012 type of food served by a restaurant or restaurant chain or national food culture
P411 Formal stage in the process of attaining sainthood per the subject's religious organization
P451 someone with whom the person is in a relationship without being married. Use "spouse" (P26) for married couples
P156 immediately following item in a series of which the subject is a part, preferably use as qualifier of P179 [if the subject has been replaced, e.g. political offices, use "replaced by" (P1366)]
P1552 inherent or distinguishing quality or feature of the entity. Use a more specific property when possible.
P3113 expected part that the item does not have (for qualities, use P6477)
P551 the place where the person is or has been, resident
P825 person or organization to whom t

In [67]:
oven_entity_set_plus = oven_entity_set.copy()
for e in tqdm(oven_entity_set):
    start = e
    flag = True
    n = 0
    while flag and n<20:
        flag = False
        n+=1
        if start in triplets_h_dict:
            for triplet in triplets_h_dict[start]:
                if triplet[1] == 'P31' or triplet[1] == 'P279':
                    start = triplet[2]
                    oven_entity_set_plus.add(triplet[2])
                    flag = True
                    break

print(len(oven_entity_set_plus))
    
        
        
        

100%|██████████| 20549/20549 [00:00<00:00, 110831.61it/s]

22815


In [78]:
entity_path = '/home/zho1rng/oven_train/dataset/wikidata5m_s/entity.txt'
rel_path = '/home/zho1rng/oven_train/dataset/wikidata5m_s/relation.txt'

with open(entity_path, 'w') as outfile:
    for e in oven_entity_set_plus:
        outfile.writelines(e + '\n')

with open(rel_path, 'w') as outfile:
    for r in sub_rel_set:
        outfile.writelines(r + '\n')
        
triplets_h_path = '/home/zho1rng/oven_train/dataset/wikidata5m_s/triplet_h.jsonl'
triplets_t_path = '/home/zho1rng/oven_train/dataset/wikidata5m_s/triplet_t.jsonl'

sub_triplets_h_dict = {}
sub_triplets_t_dict = {}
for key in triplets_h_dict:
    for triplet in triplets_h_dict[key]:
        if triplet[0] in oven_entity_set_plus and triplet[2] in oven_entity_set_plus and triplet[1] in sub_rel_set:
            if not triplet[0] in sub_triplets_h_dict:
                sub_triplets_h_dict[triplet[0]] = []
            else:
                sub_triplets_h_dict[triplet[0]].append(triplet)
            
            if not triplet[2] in sub_triplets_t_dict:
                sub_triplets_t_dict[triplet[2]] = []
            else:
                sub_triplets_t_dict[triplet[2]].append(triplet)

with open(triplets_h_path, 'w') as outfile:
    for key in sub_triplets_h_dict:
        if sub_triplets_h_dict[key] != []:
            json_object = {'key': key, 'triplets':sub_triplets_h_dict[key]}
            outfile.writelines(json.dumps(json_object) + '\n')
        
with open(triplets_t_path, 'w') as outfile:
    for key in sub_triplets_t_dict:
        if sub_triplets_t_dict[key] != []:
            json_object = {'key': key, 'triplets':sub_triplets_t_dict[key]}
            outfile.writelines(json.dumps(json_object) + '\n')



## Knowledge Base Pre-processing


In [2]:
import json
from tqdm import tqdm
with open('/home/zho1rng/oven_train/dataset/knowledge_base/Wiki6M_ver_1_0.jsonl', 'r') as infile, open('/home/zho1rng/oven_train/dataset/knowledge_base/Wiki6M_ver_1_1.jsonl', 'w') as outfile:
    for f in tqdm(infile):
        json_object = json.loads(f)
        json_object_n = {'wikidata_id': json_object['wikidata_id'], 'wikipedia_title':json_object['wikipedia_title'], 'wikipedia_summary': json_object['wikipedia_summary']}
        outfile.writelines(json.dumps(json_object_n) + '\n')

6084491it [04:57, 20466.43it/s]


In [ ]:
from tqdm import tqdm
with open('/home/zho1rng/oven_train/dataset/knowledge_base/Wiki6M_ver_1_0.jsonl', 'r') as infile, open('/home/zho1rng/oven_train/dataset/knowledge_base/Wiki6M_ver_1_1.jsonl', 'w') as outfile:
    for f in tqdm(infile):
        json_object = json.loads(f)
        json_object_n = {'wikidata_id': json_object['wikidata_id'], 'wikipedia_title':json_object['wikipedia_title'], 'wikipedia_summary': json_object['wikipedia_summary']}
        outfile.writelines(json.dumps(json_object_n) + '\n')

825it [00:00, 170189.89it/s]


## Knowledge Graph Embedding Pre-Processing

In [ ]:
extensions = ['.JPEG', '.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff']
from knowcol.datasets.utils import pil_loader

def load_image(self, img_id):
    for ext in extensions:
        try:
            img_path = self.imgid2path(img_id, ext)
            return pil_loader(img_path)
        except Exception as e:
            pass
    
    assert('No image with this id!')



In [ ]:
import json
from knowcol.datasets.utils import pil_loader
from pathlib import Path

entity_set = set()

def qid2img( qid: str):
    id = int(qid[1:])
    if id < 100:
        return (Path('/home/zho1rng/oven_train/dataset/knowledge_base') / 'wikipedia_images_full' / qid / f'{qid}.jpg').as_posix()
    else:
        return (Path('/home/zho1rng/oven_train/dataset/knowledge_base') / 'wikipedia_images_full' / qid[:4] / f'{qid}.jpg').as_posix()


files = [
        '/home/zho1rng/oven_train/dataset/oven_data/oven_entity_train.jsonl', 
         '/home/zho1rng/oven_train/dataset/oven_data/oven_entity_val.jsonl', 
         '/home/zho1rng/oven_train/dataset/oven_data/oven_query_train.jsonl', 
         '/home/zho1rng/oven_train/dataset/oven_data/oven_query_val.jsonl',
         '/home/zho1rng/oven_train/dataset/test_data/oven_entity_test.jsonl', 
         '/home/zho1rng/oven_train/dataset/test_data/oven_query_test.jsonl', 
         '/home/zho1rng/oven_train/dataset/test_data/oven_human.jsonl'
        ]

for path in files:
    with open(path, 'r') as f:
        for line in f:
            json_object = json.loads(line)
            q_id = json_object['entity_id']
            entity_set.add(q_id)

l = []
for q_id in entity_set:
    image_path = qid2img(q_id)
    if not Path(image_path).is_file():
        l.append(q_id)
print(len(l))



1157


In [7]:
!module load proxy4server-access/ & proxy_on

/bin/bash: proxy_on: command not found


In [8]:
import requests
import os

def download_wikidata_image(qid, save_path="downloaded_image.jpg"):
    """
    Download the main image (P18) of a Wikidata entity (QID) and save it locally.
    
    Args:
        qid (str): Wikidata QID (e.g., "Q42").
        save_path (str): Path to save the downloaded image.
        
    Returns:
        str: Path to the saved image or an error message.
    """
    # Step 1: Get the JSON data for the QID
    api_url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    response = requests.get(api_url)
    
    if response.status_code != 200:
        return f"Failed to fetch data for QID {qid}. HTTP Status: {response.status_code}"
    
    data = response.json()
    
    try:
        # Step 2: Extract the P18 property (image name)
        image_name = data['entities'][qid]['claims']['P18'][0]['mainsnak']['datavalue']['value']
        
        # Step 3: Construct the image URL
        image_name_encoded = image_name.replace(" ", "_")
        commons_base_url = "https://upload.wikimedia.org/wikipedia/commons"
        hash_prefix = f"{image_name_encoded[0]}/{image_name_encoded[0:2]}"
        image_url = f"{commons_base_url}/{hash_prefix}/{image_name_encoded}"
        
        # Step 4: Download the image
        img_response = requests.get(image_url, stream=True)
        
        if img_response.status_code == 200:
            with open(save_path, "wb") as f:
                for chunk in img_response.iter_content(1024):
                    f.write(chunk)
            return f"Image downloaded successfully and saved to {save_path}"
        else:
            return f"Failed to download image. HTTP Status: {img_response.status_code}"
    
    except KeyError:
        return f"No image (P18 property) found for QID {qid}."

# Example usage
qid = "Q42"  # Replace with your desired QID
save_path = "douglas_adams.jpg"  # File name to save the image
result = download_wikidata_image(qid, save_path)
print(result)

ConnectionError: HTTPSConnectionPool(host='www.wikidata.org', port=443): Max retries exceeded with url: /wiki/Special:EntityData/Q42.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f209a061fc0>: Failed to resolve 'www.wikidata.org' ([Errno -2] Name or service not known)"))

## Node Entity Preprocessing

In [ ]:
import torch
from PIL import Image
import open_clip
from knowcol.datasets.kg import KG
from knowcol.datasets.data_module import _transform
from knowcol.datasets.utils import _load_image_from_path

clip_model = 'ViT-L-14'
pretrained = 'commonpool_xl_s13b_b90k'

knowledge_base_path='dataset/knowledge_base'
entity_set_path='dataset/wikidata5m_s/entity.txt'
relation_path='dataset/wikidata5m_s/relation.txt'
triplet_h_path='dataset/wikidata5m_s/triplet_h.jsonl'
triplet_t_path='dataset/wikidata5m_s/triplet_t.jsonl'
N_px = 224
device = 'cuda'
kg = KG(
        knowledge_base_path=knowledge_base_path, 
        entity_set_path=entity_set_path, 
        relation_path=relation_path,
        triplet_h_path=triplet_h_path,
        triplet_t_path=triplet_t_path
        )

model, _, preprocess = open_clip.create_model_and_transforms(clip_model, pretrained=pretrained)
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
tokenizer = open_clip.get_tokenizer(clip_model)



entity_embs = []
entity_summaries = kg.get_ents_text(kg.entities)
entity_imgs_path = kg.get_ents_image(kg.entities)       


for i in tqdm(range(kg.n_ent)):
    with torch.no_grad():
        entity_summary_tokenized = tokenizer(entity_summaries[i]).squeeze().to(device)
        entity_image_path = entity_imgs_path[i]
        entity_image, has_entity_img = _load_image_from_path(entity_image_path)
        entity_image = _transform(N_px)(entity_image).to(device)
        
        entity_image = entity_image[None, ...]
        entity_summary_tokenized = entity_summary_tokenized[None, ...]
        entity_image_emb = model.encode_image(entity_image) # (B, N_z)
        entity_text_emb = model.encode_text(entity_summary_tokenized) # (B, N_z)
        entity_image_emb = entity_image_emb * has_entity_img + entity_text_emb * (1.0 - has_entity_img) # replace image emb. of the entities without sample image with its text emb

        entity_emb = entity_text_emb + entity_image_emb
        entity_embs.append(entity_emb)






In [ ]:
import requests
import pdb

def load_list_from_txt(file_path):
    """
    Loads a list from a text file, where each line in the file is an item in the list.

    Args:
        file_path (str): Path to the text file.

    Returns:
        list: A list of strings read from the file.
    """
    try:
        with open(file_path, 'r') as file:
            return [line.strip() for line in file]
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []

def get_wikipedia_image_by_qid(qid):
    """
    Fetches the image URL associated with a Wikidata QID.
    Args:
        qid (str): The Wikidata QID (e.g., "Q243" for the Eiffel Tower).
    Returns:
        str: The image URL or None if not found.
    """
    # Construct the API URL
    wikidata_url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36"
    }
    # Fetch the data from Wikidata
    response = requests.get(wikidata_url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        claims = data.get("entities", {}).get(qid, {}).get("claims", {})
        if "P18" in claims:  # P18 is the property for images
            image_name = claims["P18"][0]["mainsnak"]["datavalue"]["value"]
            return f"https://commons.wikimedia.org/wiki/Special:FilePath/{image_name.replace(' ', '_')}"
    return None

def download_image(image_url, save_path):
    """
    Downloads an image from a given URL.
    Args:
        image_url (str): The URL of the image.
        save_path (str): The file path to save the image.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36"
    }
    response = requests.get(image_url, headers=headers)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Image saved to {save_path}")
    else:
        print(f"Failed to download the image. Status code: {response.status_code}")

loaded_list = load_list_from_txt('qids.txt')
for qid in loaded_list:
    image_url = get_wikipedia_image_by_qid(qid)

    if image_url:
        print(f"Image URL: {image_url}")
        download_image(image_url, f"images/{qid}.jpg")
    else:
        print(f"No image found for the given QID {qid}.")